In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium
import pandas as pd
import math
import numpy as np
from haversine import haversine

In [ ]:
df = pd.read_csv('/content/data_1.csv')
coordinates = df.values.tolist()

In [ ]:
city_coords = (12.938655, 77.581057)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'black']
n = len(colors)

In [ ]:
map = folium.Map(location=city_coords, zoom_start=15, control_scale = True)
for latitude, longitude in coordinates:
    folium.Marker(location=(latitude, longitude), icon=folium.Icon()).add_to(map)
map

In [ ]:
def cluster_by_distance(points, radius):
    distances = []
    n = len(points)
    for i in range(n):
        distances.append([])
        for j in range(i + 1, n):
            distances[-1].append(haversine(points[i], points[j])) #unit is km

    # visited = [0]*n
    clusters = []
    for i in range(n):
        clusters.append([])
        count = 0
        for d in distances[i]:
            count += 1
            if d <= radius:
                clusters[-1].append(points[i+count])

    return clusters


In [ ]:
radius, min_samples = 0.2, 15
clusters_1 = [x for x in cluster_by_distance(coordinates.copy(), radius) if len(x) >= min_samples]
map1 = folium.Map(location=city_coords, zoom_start=15)
for i in range(len(clusters_1)):
    cluster = clusters_1[i]
    color = colors[i % n]
    for point in cluster:
        x, y = point[0], point[1]
        folium.Marker(location= (x, y), icon=folium.Icon(color=color)).add_to(map1)
map


In [ ]:
#https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
epsilon = 0.5 / 6371.0
min_samples = 15
db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(df.to_numpy()))
labels = db.labels_
map2 = folium.Map(location=city_coords, zoom_start=15, control_scale = True)
for i in range(len(labels)):
    if labels[i] == -1:
        color = 'gray'
        folium.Marker(location= coordinates[i], icon=folium.Icon(color=color)).add_to(map2)

    else:
        color = colors[labels[i] % n]
        folium.Marker(location= coordinates[i], icon=folium.Icon(color=color)).add_to(map2)
map2